In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt 
import time
# 查询系统可用的 GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
# 确保有可用的 GPU 如果没有, 则会报错
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
# 设置参数,该段务必在运行jupyter的第一段代码执行，否则会无法初始化成功
# 仅在需要时申请显存空间（程序初始运行时消耗很少的显存，随着程序的运行而动态申请显存）
tf.config.experimental.set_memory_growth(physical_devices[0], True)

#### 1.数据处理
- 数据shuffle
- TFRecord方式
- TextLineDataset方式
- from_generator方法

![通过对大数据进行拆分成小批量数据的过程](./markdown_pics/通过对大数据进行拆分成小批量数据的过程.png)

In [ ]:
s = Simplified('./data')
NCSVS = 100

 # 每个文件都分成100份，然后将每个文件的1/100的数据写入内存,这样就能确保内存不会溢出
categories = s.list_all_categories()
print(len(categories))

for y, cat in tqdm(enumerate(categories)):
    df = s.read_training_csv(cat)
    df['y']  = y
    df['cv'] = (df.key_id // 10 ** 7) % NCSVS  
    
    for k in range(NCSVS):
        filename = f'./shuffle_data/train_k{k}.csv'
        chunk = df[df.cv == k]
        chunk = chunk.drop(['key_id'], axis = 1)

        if y == 0 :
            chunk.to_csv(filename, index = False)
        else:
            chunk.to_csv(filename, mode = 'a', header = False, index = False)

#### 总结一下小批量读取大数据的形式：
##### 1. 对本赛题的340个文件，每一个，都按照100等份进行拆分，拆分后的文件分别有3400个
##### 2.然后对每个类别的文件的前1/100的数据做合并，然后按照标签做随机化，房子模型读取的数据趋同
##### 3.对随机化后的340个小文件做合并并做压缩，然后作为模型的第一部分的小数据输入，这样就解决内存溢出问题
##### 4.接着是模型重复的输入第2/100的数据，以此类推。模型一共需要迭代100次才能输入全部的数据。
##### 5.可以通过上图查看到最终的文件样式和对应的编号

#### TFRecord方式
- 按照上图的文件类型分割后，仅拿train_k0.csv.gz文件而言，生成的tfrecord的文件大约为7.7GB。
- 对于所有的train_k{}csv.gz文件，这样算下来，大概为770GB，因此需要消耗巨大的硬盘空间。

In [ ]:
with tf.io.TFRecordWriter(tfrecord_file) as writer:
    for filename in fileList[:1]:
        df = pd.read_csv(filename)
        df['drawing'] = df['drawing'].apply(json.loads)

        for row in range(df.shape[0]):
            drawing = df.loc[row, 'drawing']
            img = draw_cv2(drawing, BASE_SIZE = 128, size = 128, lw = 6) # 构建一个图像
            img = img.tostring()
            label = df.loc[row, 'y']

        # 建立tf.train.Feature 字典
        feature = {
            'image': tf.train.Feature(bytes_list = tf.train.BytesList(value = [img])),  # 图片是一个 Bytes 对象
            'label': tf.train.Feature(int64_list = tf.train.Int64List(value = [label])) # 标签是一个 Int 对象
        }
        example = tf.train.Example(feature = tf.train.Features(feature = feature)) # 通过字典建立Example
        writer.writer(example.SerializePartialToString()) # 将Example序列化并写入TFRecord文件

#### TextLineDataset方式

In [ ]:
def draw_cv2(raw_strokes, size = 64, lw = 6):
    raw_strokes = eval(raw_strokes.numpy())
    img = np.zeros((256,256), np.uint8)
    
    for stroke in raw_strokes:
        for i in range(len(stroke[0]) - 1):
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]), (stroke[0][i+1], stroke[1][i+1]), 255, lw)
    return cv2.resize(img, (size, size))

In [ ]:
def tf_draw_cv2(image, label):
    [image] = tf.py_function(draw_cv2, [image], [tf.float32])
    image = tf.reshape(image, (64,64,1))
    label = tf.one_hot(label, depth = NCATS)
    image.set_shape((64, 64, 1))
    label.set_shape((340,))
    return image, label

In [ ]:
train_ds = tf.data.TextLineDataset(fileList[2], compression_type= 'GZIP').skip(1).map(parse_csv, num_parallel_calls= tf.data.experimental.AUTOTUNE)

train_ds = train_ds.map(tf_draw_cv2, num_parallel_calls= tf.data.experimental.AUTOTUNE)
train_ds = train_ds.prefetch(buffer_size = tf.data.experimental.AUTOTUNE).shuffle(3000).batch(1024)

![py_function的作用](./markdown_pics/py_function的作用.png)

![py_function的解释](./markdown_pics/py_function的解释.png)

#### from_generator方法
- tf.data.Dataset类提供from_generator方法，针对大数据量比较友好，难点是gen必须是一个可调用的对象，返回支持iter()对象的协议

In [9]:
import itertools

def gen():
    for i in itertools.count(start = 1):
        print(i)
        yield(i, [1] * i)

dataset = tf.data.Dataset.from_generator(
    gen,
    (tf.int64, tf.int64),
    (tf.TensorShape([]), tf.TensorShape([None]))
)

list(dataset.take(3).as_numpy_iterator())

1
2
3


[(1, array([1])), (2, array([1, 1])), (3, array([1, 1, 1]))]

In [3]:
def foo():
    print("starting...")
    while True:
        res = yield 4
        print("res:",res)

In [4]:
g = foo() # 执行函数调用后，函数没有输出任何内容

In [5]:
print(next(g)) # 结合next的时候，才开始生效, 且yield作为return的作用返回了4，但是第二个print并没有生效

starting...
4


In [7]:
print("*"*20) 
print(next(g)) #从这里可以看出，再一次迭代后，开始执行的是yield后面的语句，相当于接着上面一个执行未执行完毕的部分，但是res并没拿到 4这个值
# 然后执行完毕，又开始执行yield的return部分，然后执行又结束了

********************
res: None
4


![from_generator方法参数](./markdown_pics/from_generator方法参数.png)

In [11]:
import pandas as pd
class Dataloader(object):
    def __init__(self, resize_height =64, resize_width = 64, batch_size = 512, fileList = None, size = 256, lw = 6):
        self.resize_height = resize_height # 图片的高度
        self.resize_width = resize_width # 图片的宽度
        self.batch_size = batch_size # batch大小
        self.fileList = fileList # 文件数据
        self.size = size # 画图时图片的大小
        self.lw = lw

    def __call__(self):
        def _generator(size, lw):
            while True:
                for filename in np.random.permutation(self.fileList):
                    df = pd.read_csv(filename)
                df['drawing'] = df['drawing'].apply(json.loads) # 转换一下数据格式
                x = np.zeros((len(df), size, size))

                for i, raw_strokes in enumerate(df.drawing.values):
                    x[i] = draw_cv2(raw_strokes, size =size, lw = lw)
                
                x = x/ 255.0
                x = x.reshape((len(df), size, size,1)).astype(np.float32)  #这里转换为黑白图像
                y = tf.keras.utils.to_categorical(df.y, num_classes= n_labels)

                for x_i, y_i in zip(x,y):
                    yield(x_i, y_i)



                dataset = tf.data.Dataset.from_generator(generator = _generator,
                    output_types=(tf.dtypes.float32, tf.dtypes.float32),
                    output_shapes=((self.resize_height, self.resize_width, 1), (340, )),
                    args=(self.size, self.lw)
                )

                dataset = dataset.prefetch(buffer_size = 10240)
                dataset = dataset.shuffle(buffer_size = 10240).batch(self.batch_size)

                return dataset

#### 2.建模方法
- 预训练模型

![预训练的建模方法](./markdown_pics/预训练的建模方法.png)

##### 针对不同的场景，使用不同的预训练模型

![预训练模型的列表](./markdown_pics/预训练模型的列表.png)

![预训练模型调用的方法举例](./markdown_pics/预训练模型调用的方法举例.png)

![预训练模型的详细参数1](./markdown_pics/预训练模型的详细参数1.png)

![预训练模型的详细参数2](./markdown_pics/预训练模型的详细参数2.png)

#### 3.Baseline构建
- 数据处理
- 数据读取
- 建模方法
- 参数调优

![卷积神经网络baseline构建的要点](./markdown_pics/卷积神经网络baseline构建的要点.png)